In [1]:
# %%
import pandas as pd
import numpy as np
from dtaidistance import dtw
import os
from sklearn.cluster import SpectralClustering
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from __future__ import absolute_import
from __future__ import print_function
from GPUDTW import cuda_dtw, cpu_dtw, dtw_1D_jit2
import numpy
import time


In [2]:
# %%
os.getcwd()


'/home/maxime/Documents/WORKSPACES/forecasting_models/cluster/features_clustering'

In [3]:
# %%
data = pd.read_csv('../../tests/enc_X_train.csv')


In [4]:
def calculate_dtw_distance_matrix(df):
    # Subsample the time series if needed (GPUDTW has a limit around 2500 points)
    if df.shape[0] > 1000:
        print('input sequence too long')
        df = df.iloc[:1000, :]  # Take every other point
    
    # Convert and transpose
    features = df.values.T.astype(np.float32)
    
    # Compute distances
    distances = cuda_dtw(features, features)
    
    return pd.DataFrame(distances, columns=df.columns, index=df.columns)


In [5]:
# Let's verify the data first
print("DataFrame columns:", len(data.columns))
print("DataFrame rows:", len(data))

# Then compute the distances
dtw_matrix = calculate_dtw_distance_matrix(data)


DataFrame columns: 54
DataFrame rows: 1153
input sequence too long
{'Max_Grid_X': 2147483647, 'Max_Grid_Y': 65535, 'Max_Threads_pre_Block': 1024, 'Max_share_memeory_per_block': 49152, 'COMPUTE_CAPABILITY_MAJOR': 8, 'total_memory': 8321171456}
grids,block_DimX 1034420 1 1 1000
Splits [0, 54]
grid_run T0,Gx,Gy,Tbits  54 54 1 0.20599365234375


ValueError: Shape of passed values is (54, 53), indices imply (54, 54)

In [ ]:
# %%


def distance_to_similarity_gaussian(distance_matrix, sigma=1.0):
    # Transformation en similarité avec une fonction gaussienne
    similarity_matrix = np.exp(-distance_matrix**2 / (2 * sigma**2))
    # Similitude maximale sur la diagonale
    np.fill_diagonal(similarity_matrix.values, 1)
    return similarity_matrix



In [ ]:
# %%
similarity_matrix_gaussian = distance_to_similarity_gaussian(dtw_matrix)
similarity_matrix_gaussian


In [ ]:
# %%


def clustering_with_spectral(similarity_matrix, n_clusters=3):
    spectral = SpectralClustering(
        n_clusters=n_clusters, affinity='precomputed')
    labels = spectral.fit_predict(similarity_matrix)
    return labels



In [ ]:
# %%
labels_spectral = clustering_with_spectral(similarity_matrix_gaussian)
labels_spectral


In [ ]:
# %%


def tsne_visualization(similarity_matrix):
    # Conversion de la matrice de similarité en matrice de distance
    distance_matrix = 1 - similarity_matrix
    tsne = TSNE(metric='precomputed', n_components=2, perplexity=30)
    tsne_result = tsne.fit_transform(distance_matrix)
    return tsne_result



In [ ]:
# %%
# Visualisation t-SNE
tsne_result = tsne_visualization(similarity_matrix_gaussian)
# `labels` peut venir du clustering
plt.scatter(tsne_result[:, 0], tsne_result[:, 1], c=labels)
plt.title('t-SNE Visualisation based on Similarity Matrix')
plt.show()
